In [2]:
import sqlite3
import numpy as np

In [1]:
basename = "../outputs/plateau"
sql_name = basename + ".sql"

In [3]:
try: 
    conn = sqlite3.connect(":memory:")
    conn.executescript(open(sql_name).read())
except OSError:
    print("IMPORT ERROR: Could not import {}".format(sql_name))

OperationalError: no such column: nan

In [4]:
c = conn.cursor()


c.execute('select  min(st.Z), max(st.Z), su.FULL_NAME, ct.TOPO_DATE\
          from STATION st, SURVEY su, CENTRELINE ct\
          WHERE st.SURVEY_ID = su.ID and st.NAME !="." and st.NAME !="-" and ct.SURVEY_ID = su.ID  and ct.TOPO_DATE > "2021-12-31"\
          GROUP BY su.FULL_NAME ORDER BY ct.TOPO_DATE ;')


In [5]:
z_ranges = c.fetchall()

In [60]:
dates = [i[3] for i in z_ranges]
alt_diff = [i[1]-i[0] for i in z_ranges]
names = [i[2].split(".")[0] for i in z_ranges]

In [61]:
names

['styx',
 'echo_falls',
 'novi_vhod',
 '100_cocoa',
 'coda',
 'the_expo_strikes_back',
 'the_deep_awakens',
 'cantina_canyon',
 'bin_bag_palace',
 'sisyphus_slope',
 'mudaholic',
 'cantina_canyon_2',
 'antares',
 'who_dies_first',
 'the_aviary',
 'blue_lagoon',
 'delta_blues',
 'neu_alliance',
 'auld_alliance_resurvey',
 'fools_gold',
 'a_dry_place',
 'blossom_hill']

In [19]:
c.execute('select sum(sh.ADJ_LENGTH), su.FULL_NAME, ct.TOPO_DATE, shf.FLAG, su.ID\
          from SHOT sh, SURVEY su, CENTRELINE ct, STATION stn1, STATION stn2, SHOT_FLAG shf\
          WHERE sh.TO_ID = stn1.ID and sh.TO_ID = stn2.ID and stn2.NAME != "." and  stn2.NAME != "-" and stn1.SURVEY_ID = su.id and ct.SURVEY_ID = su.ID  and ct.TOPO_DATE > "2021-12-31" and shf.SHOT_ID = sh.ID and shf.SHOT_ID = sh.ID\
          GROUP BY su.FULL_NAME ORDER BY ct.TOPO_DATE;')


In [20]:
duplicates = c.fetchall()

In [34]:
dpl_survey_ids =np.array([i[4] for i in duplicates])
dpl_lengths = np.array([i[0] for i in duplicates])

In [26]:
c.execute('select sum(sh.ADJ_LENGTH), su.FULL_NAME, ct.TOPO_DATE, su.ID\
          from SHOT sh, SURVEY su, CENTRELINE ct, STATION stn1, STATION stn2\
          WHERE sh.TO_ID = stn1.ID and sh.TO_ID = stn2.ID and stn2.NAME != "." and  stn2.NAME != "-" and stn1.SURVEY_ID = su.id and ct.SURVEY_ID = su.ID  and ct.TOPO_DATE > "2021-12-31"\
          GROUP BY su.FULL_NAME ORDER BY ct.TOPO_DATE;')

In [27]:
raw = c.fetchall()

In [42]:
raw_lengths = np.array([i[0] for i in raw])
raw_ids = np.array([i[3] for i in raw])

duplicate_lengths = np.zeros(len(raw_lengths))

In [43]:
duplicate_lengths[np.isin(raw_ids, dpl_survey_ids)] =  dpl_lengths

In [44]:
new_lengths = raw_lengths - duplicate_lengths

In [62]:
clean = np.array([names, dates, alt_diff, new_lengths, duplicate_lengths], dtype=str).T

In [68]:
with open("last_surveyed_since_2022-01-01.txt", "w") as f:
    f.write("name, date, vertical extent, length, duplicate length,\n")
    for line in clean:
        for element in line:
            try:
                element = float(element)
                f.write(f"{element:.3f}")
            except:
                f.write(str(element))
            f.write(",")
        f.write("\n")
    f.close()

In [4]:
c.close()

NameError: name 'c' is not defined